# The Traveling Baseball Fan Problem

$$
\begin{array}{rrcll}
\text{minimize:} & \text{Total Time} \\
\text{subject to:} & \text{Visit all stadiums once} \\
& \text{Balance}
\end{array}
$$

$$
\begin{array}{rrcll}
\text{minimize:} & \displaystyle \sum_{(i,j) \in ARCS} c_{i,j} x_{i,j} \\
\text{subject to:} &  \displaystyle \sum_{(i,j) \in ARCS \; | \; loc[j] = s} x_{i,j} &=& 1 & & \forall s \in STADIUMS \\
& \displaystyle \sum_{(i,g) \in ARCS} x_{i,g} - \sum_{(g,j) \in ARCS} x_{g,j} & = & 0 & & \forall g \in NODES
\end{array}
$$

# Solutions

In [1]:
import os

solutions = []
for filename in sorted(os.listdir(os.getcwd() + '/results'), reverse=True):
    f = open('results/'+filename, 'r')
    row = {'ref': filename}
    for line in f:
        if 'schd' in line:
            row['schd'] = []
            for visit in f:
                if ']' not in visit:
                    items = visit.split(',')
                    row['schd'].append(items)
        elif ': ' in line:
            pl = line.split(': ')
            row[pl[0]] = pl[1].replace('\n','')
    solutions.append(row)

In [13]:
solutions = sorted(solutions, key=lambda x: (x['mont'], x['sdat'], x['objt']))
linef = '{:>2} {:>2} {:>11} {:>5} {:>4} {:>13} {:>12} {:>16} {:>12}'

df_source = []

for i, v in enumerate(solutions):
    period = v['sdat'][5:] + '/' + v['edat'][5:]
    df_source.append([
        i+1, v['ref'].split('.')[0], float(v['objt']), period, float(v['mont']), float(v['vars']), float(v['cons']),
        float(v['solv'].split()[0]), float(v['time'].split()[0]),
        float(v['dist'].split()[0]), float(v['cost'].split()[0])
    ])

# Draw all maps
import folium
from dateutil.parser import parse

allplots = []
allhtml = []
for i in solutions:
    route = i['schd']
    tbfmap = folium.Map(location=[39.82, -98.58], zoom_start=4, tiles="OpenStreetMap")
    for node in route:
        popup_text = '''
        Game {}: {} @ {}<br>
        {}, {}<br>
        {}            
        '''.format(node[0], node[2], node[3], node[1], node[4],
                   parse(node[5]).strftime("%A, %B %d, %Y - %I:%M %p"))
        folium.Marker(location=[float(node[6]), float(node[7])],popup=popup_text,
                      #icon=folium.Icon(icon='adjust', prefix='fa')
                      icon=folium.DivIcon(html='<i class="fa fa-map-pin fa-stack-2x" style="font-size:28px"></i><strong style="text-align: center; color: white; font-family: Helvetica Neue, Helvetica, Arial; font-size:12px; width:16px;" class="fa-stack-1x">{}</strong>'.format(node[0]))
                      ).add_to(tbfmap)
    
    lines = folium.PolyLine(locations=[(float(i[6]), float(i[7])) for i in route])
    lines.add_to(tbfmap)
    allplots.append(tbfmap)
    allhtml.append(tbfmap._repr_html_())
    
maps=ColumnDataSource(data=dict(map= allhtml))

In [26]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models.widgets import Slider, Select
from bokeh.models import HoverTool, ColumnDataSource, TapTool, Div
from bokeh.io import curdoc
from bokeh.layouts import widgetbox, column, row
import pandas as pd
import random
from bokeh.embed import components
from IPython.core.display import display, HTML
from bokeh.models.callbacks import CustomJS



import time
from bokeh.events import Tap

output_notebook()

df = pd.DataFrame(
    df_source,
    columns=[
        'id', 'ref', 'objt', 'period', 'mont', 'vars', 'cons', 'solv',
        'time', 'dist', 'cost'
    ]).set_index(['id'])

axis_map = {
    "Tour Length": 'tour', "Tour Distance": 'dist', "Tour Cost": 'cost', "Period Length": 'mont'
}

df2=df.rename(columns = {'ref': 'Reference', 'objt':'Obj. Type', 'period': 'Period', 'mont': 'P.Length', 'vars': '# Variables', 'cons': '# Constraints',
                        'solv': 'Solve Time', 'time': 'Tour Time', 'dist': 'Tour Distance', 'cost': 'Tour Cost'})
display(HTML(df2.to_html(index_names=False)))



Loading BokehJS ...

,Reference,Obj. Type,Period,P.Length,# Variables,# Constraints,Solve Time,Tour Time,Tour Distance,Tour Cost
1,140412600622048,0.0,04-01/06-01,2.0,22831.0,823.0,143.899,24.976,19988.0,5245.640
2,140412080713008,1.0,04-01/06-01,2.0,22831.0,823.0,476.705,26.248,15480.0,4960.229
3,140412102239344,0.0,06-01/08-01,2.0,22446.0,800.0,175.492,25.743,21503.0,5496.897
4,140412146831880,1.0,06-01/08-01,2.0,22446.0,800.0,280.260,27.250,13027.0,4845.200
5,140412110375960,0.0,08-01/10-01,2.0,23089.0,839.0,93.418,25.125,21102.0,5376.450
6,140412072783824,1.0,08-01/10-01,2.0,23089.0,839.0,213.208,25.378,15129.0,4812.101
7,140412117171448,0.0,04-01/07-01,3.0,35468.0,1222.0,394.426,24.854,19967.0,5227.742
8,140412071868064,1.0,04-01/07-01,3.0,35468.0,1222.0,726.213,26.250,12996.0,4712.100
9,140411979803896,0.0,07-01/10-01,3.1,34590.0,1210.0,117.687,25.125,20508.0,5317.050
10,140412007400000,1.0,07-01/10-01,3.1,34590.0,1210.0,544.462,25.378,15129.0,4812.101


In [56]:

callback = CustomJS(code="""
console.log('Tap event occured at x-position: ' + cb_obj.x + ',' + cb_obj.y)
""")
    
def add_plots():
    source=ColumnDataSource(df)
    div = Div(width=900, height=600, text='<h2>Solution: 1</h2>Length: ' + str(df.iloc[0, 7]) + ' days, Distance: ' + str(df.iloc[0, 8]) + ' miles' + allhtml[0])

    hover = HoverTool(tooltips=[
        ("ID", "@id"),
        ("Tour Time", "@time days"),
        ("Tour Distance", "@dist miles"),
        ("Tour Cost", "@cost USD"),
        ("Obj. Type", "@objt"),
        ("Period", "@mont months"),
        ("Num of Variables", "@vars"),
        ("Num of Constraints", "@cons"),
        ("Solve Time", "@solv secs"),
    ])

    plots = [None]*4
    plot_pairs = [
        ('dist', 'time'),
        ('dist', 'cost'),
        ('solv', 'vars'),
        ('solv', 'time')    
    ]
    plot_labels = [
        ('Tour Distance (miles)', 'Tour Time (days)'),
        ('Tour Distance (miles)', 'Tour Cost (USD)'),
        ('Solve Time (secs)', 'Number of Variables'),
        ('Solve Time (secs)', 'Tour Time (days)')
    ]
    
    for i, pp in enumerate(plot_pairs):
        
        hover = HoverTool(tooltips=[
            ("ID", "@id"),
            (plot_labels[i][0], "@{}{{0.0}}".format(pp[0])),
            (plot_labels[i][1], "@{}{{0.0}}".format(pp[1]))
        ])
        
        plots[i] = figure(plot_width=450, plot_height=300,
                          tools=[hover, 'save', 'pan', 'tap', 'box_zoom', 'reset']
                         )
        c = plots[i].circle(
                     x=pp[0], y=pp[1], 
                     source=source, size=15,
                     hover_fill_color="pink",
                     selection_fill_color="green",
        )
        plots[i].xaxis.axis_label = plot_labels[i][0]
        plots[i].yaxis.axis_label = plot_labels[i][1]
        taptool = plots[i].select(type=TapTool)
        code = '''
        var s = source.selected['1d'].indices[0];
        div.text = '<h2>Solution: ' + (s+1) + '</h2>Length: ' + source.data.time[s] + ' days, Distance: ' + source.data.dist[s] + ' miles' + maps.data.map[s];
        '''
        taptool.callback = CustomJS(args=dict(source=source, maps=maps, 
                                              div=div
                                             ), code=code)

    #script, div = components(column(row(plots[0], plots[1]),row(plots[2], plots[3])))
    display(HTML('<h1>Plots</h1>'))
    show(column(row(plots[0], plots[1]),row(plots[2], plots[3]), div), notebook_handle=True)
    #display(HTML('<h1>Map</h1><b>Solution: 1</b>'))
    #show(div)
    

def add_map_plot(i):
    display(HTML('<h1>Map</h1>'))
    display(HTML(allhtml[i]))
    #print(allhtml[i])
    #display(HTML(allhtml[i]))

def change_map(id):
    print(id)    
    pass
    
add_plots()
#add_map_plot(0)


In [ ]:
display(HTML(allhtml[11]))